In [1]:
%pwd  #look at the current work dir
%cd  ..

/home/astro/Documents/UvA/Block 5 - ATCS/ATCS


In [18]:
import torch
from torchtext.legacy.datasets.nli import SNLI
from torchtext.legacy.data import Field
from torchtext.legacy import data
import torchtext
from torch.utils.data import DataLoader

In [20]:
TEXT = Field(lower=True, include_lengths=False, batch_first=True,tokenize='spacy',tokenizer_language="en_core_web_sm")
LABEL = Field(sequential=False)


['snli_1.0_train.jsonl',
 'snli_1.0_train.txt',
 'snli_1.0_test.jsonl',
 'snli_1.0_dev.txt',
 'snli_1.0_dev.jsonl',
 'snli_1.0_test.txt']

In [28]:
#Read the snli data and rewrite it with only 1000 top so is easier to load it and iterate coding
from os import listdir
from os.path import isfile, join
path_data = "./data/snli/snli_1.0/"
onlyfiles = [f for f in listdir(path_data) if (isfile(join(path_data, f)) and 'snli' in f)]
print(onlyfiles)
max_lines = 50000
small_data_path = "./subdata/snli/snli_1.0/"
if not os.path.exists(small_data_path):
    os.makedirs(small_data_path)
print("Writing files with up to",max_lines," max lines in path:")
print(small_data_path)
for f in onlyfiles:

    max_lines = 50000 if 'train' in f else 10000
    with open(path_data+f) as myfile:
        subset_data = [next(myfile) for x in range(max_lines)]
    with open(small_data_path+f, 'w') as new_f:
        for item in subset_data:
            new_f.write(item)

['snli_1.0_train.jsonl', 'snli_1.0_train.txt', 'snli_1.0_test.jsonl', 'snli_1.0_dev.txt', 'snli_1.0_dev.jsonl', 'snli_1.0_test.txt']
Writing files with up to 50000  max lines in path:
./subdata/snli/snli_1.0/


In [21]:
# make splits for data
train, dev, test = SNLI.splits(TEXT, LABEL, root= './subdata')


In [ ]:
train, dev, test = SNLI.splits(TEXT, LABEL, root= './subdata')

glove = torchtext.vocab.GloVe(name='840B', dim=300)
TEXT.build_vocab(train, vectors=glove)

In [6]:
TEXT.build_vocab(train, vectors=glove)
LABEL.build_vocab(train, specials_first=False)
n_vocab = len(TEXT.vocab.itos)

In [29]:
# n_vocab
import torch
torch.device
# TEXT.vocab.stoi

torch.device

In [7]:
import torch

import pytorch_lightning as pl
from torch import nn
from torch.nn import functional as F
from modules.AverageEmbeddings import AverageEmbeddings
from torchmetrics import MetricCollection, Accuracy, Precision, Recall
import wandb
metrics = MetricCollection([Accuracy()])

In [13]:
metrics = 
dict(metrics)

{'Accuracy': Accuracy()}

In [8]:
train_iter, dev_iter, test_iter = data.BucketIterator.splits(
    (train, dev, test), batch_sizes=(16, 256, 256), device=0)

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


In [9]:
next(iter(train_iter))


[torchtext.legacy.data.batch.Batch of size 16 from SNLI]
	[.premise]:[torch.LongTensor of size 16x30]
	[.hypothesis]:[torch.LongTensor of size 16x17]
	[.label]:[torch.LongTensor of size 16]

In [35]:
%load_ext autoreload
%autoreload 2
from pytorch_lightning import Trainer
from modules.AverageEmbeddings import AverageEmbeddings
from modules.Classifier import Classifier

model = Classifier(emb_vec=TEXT.vocab.vectors)
trainer = Trainer()
trainer.fit(model, train_iter,dev_iter)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

  | Name          | Type              | Params
----------------------------------------------------
0 | train_metrics | MetricCollection  | 0     
1 | valid_metrics | MetricCollection  | 0     
2 | emb_vec       | Embedding         | 10.1 M
3 | model         | AverageEmbeddings | 0     
4 | classifier    | Sequential        | 616 K 
5 | loss          | CrossEntropyLoss  | 0     
6 | softmax       | Softmax           | 0     
----------------------------------------------------
616 K     Trainable params
10.1 M    Non-trainable params
10.7 M    Total params
42.828    Total estimated model params size (MB)
Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

AttributeError: 'Classifier' object has no attribute 'accuracy'

In [43]:
%load_ext autoreload
%autoreload 2
from modules.Classifier import Classifier
from pytorch_lightning import LightningModule

from pytorch_lightning import Trainer
from modules.AverageEmbeddings import AverageEmbeddings
from modules.Classifier import Classifier

model = Classifier()

# train, dev, test = SNLI.splits(TEXT, LABEL, root= './subdata')

# glove = torchtext.vocab.GloVe(name='840B', dim=300)
# TEXT.build_vocab(train, vectors=glove)
checkpath = "trained_models/lstm/elated-waterfall-212/lstm.ckpt"
checkpath = 'trained_models/awe/gold/awe.ckpt'
pretrained_model = model.load_from_checkpoint(checkpath, model_name='awe',disable_nonlinear=True)

trainer = Trainer(fast_dev_run=True)
trainer.fit(model, train_iter,dev_iter)
# trainer.fit(model, train_iter,dev_iter)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
Running in fast_dev_run mode: will run a full train, val and test loop using 1 batch(es).
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


NameError: name 'train_iter' is not defined

In [44]:
pretrained_model

Classifier(
  (emb_vec): Embedding(33635, 300)
  (model): AverageEmbeddings()
  (classifier): Sequential(
    (0): Linear(in_features=1200, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=512, bias=True)
    (2): Linear(in_features=512, out_features=3, bias=True)
  )
  (loss): CrossEntropyLoss()
  (softmax): Softmax(dim=1)
  (train_metrics): MetricCollection(
    (Accuracy): Accuracy()
  )
  (valid_metrics): MetricCollection(
    (Accuracy): Accuracy()
  )
  (train_acc): Accuracy()
  (valid_acc): Accuracy()
  (test_acc): Accuracy()
)

In [18]:
import torch
torch.cuda.is_available()

True

In [27]:
from pytorch_lightning.callbacks import ModelCheckpoint
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    # dirpath='checkpoints/',
    filename='checkpoint-{epoch:02d}-{val_loss:.2f}',
    save_top_k=3,
    mode='min',
)

trainer = Trainer(callbacks=[checkpoint_callback],auto_select_gpus=True,fast_dev_run=False)

trainer.fit(model, train_iter,dev_iter)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name          | Type              | Params
----------------------------------------------------
0 | train_metrics | MetricCollection  | 0     
1 | valid_metrics | MetricCollection  | 0     
2 | emb_vec       | Embedding         | 10.1 M
3 | model         | AverageEmbeddings | 0     
4 | classifier    | Sequential        | 616 K 
5 | loss          | CrossEntropyLoss  | 0     
6 | softmax       | Softmax           | 0     
----------------------------------------------------
616 K     Trainable params
10.1 M    Non-trainable params
10.7 M    Total params
42.828    Total estimated model params size (MB)
Epoch 0:   1%|          | 200/34375 [00:02<06:41, 85.07it/s, loss=0.696, v_num=50]


1

In [ ]:

from modules.Classifier import Classifier
from pytorch_lightning import LightningModule

from pytorch_lightning import Trainer
from modules.AverageEmbeddings import AverageEmbeddings
from modules.Classifier import Classifier

# model = Classifier(emb_vec=TEXT.vocab.vectors)
pretrained_model = model.load_from_checkpoint("lightning_logs/version_44/checkpoints/epoch=0-step=7257.ckpt",emb_vec=TEXT.vocab.vectors)

trainer = Trainer(fast_dev_run=True)
trainer.fit(model, train_iter,dev_iter)
# trainer.fit(model, train_iter,dev_iter)

In [51]:
# proto = '-fast' if args.prototype else '-full'
# path = args.save_results + args.model +proto + '.json'
import json
path= 'results_senteval/awe-fast.json'
with open(path) as file:
    results = json.load(file) # use `json.loads` to do the reverse
results.keys()

{'MR': {'devacc': 77.75, 'acc': 77.12, 'ndev': 10662, 'ntest': 10662},
 'CR': {'devacc': 80.03, 'acc': 78.46, 'ndev': 3775, 'ntest': 3775},
 'MPQA': {'devacc': 87.55, 'acc': 87.66, 'ndev': 10606, 'ntest': 10606},
 'SUBJ': {'devacc': 91.63, 'acc': 91.22, 'ndev': 10000, 'ntest': 10000},
 'SST2': {'devacc': 79.36, 'acc': 79.68, 'ndev': 872, 'ntest': 1821},
 'TREC': {'devacc': 73.29, 'acc': 82.6, 'ndev': 5452, 'ntest': 500},
 'MRPC': {'devacc': 72.62,
  'acc': 64.7,
  'f1': 69.75,
  'ndev': 4076,
  'ntest': 1725},
 'SICKEntailment': {'devacc': 70.0, 'acc': 75.34, 'ndev': 500, 'ntest': 4927}}

In [57]:
def compute_macro_score(results):
    total_dev_acc = 0
    len_metrics = len(results.keys())
    for k in results.keys():
        devacc = results[k]['devacc']
        total_dev_acc += devacc
    return total_dev_acc/len_metrics
def compute_micro_score(results):
    total_dev_acc = 0
    total_samples = 0
    len_metrics = len(results.keys())
    for k in results.keys():
        devacc = results[k]['devacc']*results[k]['ndev']
        total_samples += results[k]['ndev']
        total_dev_acc += devacc
    len_metrics*=total_samples
    return total_dev_acc/total_samples

In [59]:
compute_macro_score(results)

79.02875

In [58]:
compute_micro_score(results)

82.18264305770194